#Paquetes necesarios

In [ ]:
import cv2  
import math 
import csv
import numpy as np
from ultralytics import YOLO
import pytesseract
# import easyocr

In [3]:
# Configuración de Tesseract para OCR
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
# reader = easyocr.Reader(['en', 'es'])

# Cargar modelos
model_deteccion = YOLO("yolo11n.pt")  # Modelo YOLO para detectar personas y vehículos
model_matriculas = YOLO("runs/detect/train/weights/best.pt")  # Modelo para detectar matrículas

# Nombre del archivo de video
filename = "C0142.mp4"

# Abrir archivo CSV para guardar los resultados
csv_file = open('detecciones.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['fotograma', 'tipo_objeto', 'confianza', 'identificador_tracking', 'x1', 'y1', 'x2', 'y2', 'matrícula_en_su_caso', 'confianza_matricula', 'mx1', 'my1', 'mx2', 'my2', 'texto_matricula'])

# Abrir el video
cap = cv2.VideoCapture(filename)

# Configuración del video de salida
output_filename = "video_con_detecciones.mp4"
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para formato MP4
out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

conteo_clases = {}  # Conteo total por clase
objetos_contados = {}  # Diccionario para rastrear IDs ya contados por clase
frame_count = 0
tracking_ids = {}  # Para gestionar los IDs de seguimiento
inactive_threshold = 5  # Número de fotogramas para mantener un ID inactivo
detected_ids = set()  # Conjunto para rastrear IDs ya detectados en este fotograma

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detección de vehículos y personas
    results = model_deteccion.track(frame, stream=True, persist=True, verbose=False)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Extraer coordenadas de la caja delimitadora
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Clase y confianza de la detección
            cls = int(box.cls[0])
            confianza = math.ceil((box.conf[0] * 100)) / 100
            tipo_objeto = model_deteccion.names[cls]  # Obtener la etiqueta de clase

            # ID de seguimiento del objeto
            tracking_id = str(int(box.id[0].tolist())) if box.id is not None else ''
            
            # Solo procesar si hay un tracking_id
            if tracking_id:
                # Agregar el ID al conjunto de IDs detectados
                detected_ids.add(tracking_id)

                # Contar el objeto solo si no ha sido contado ya
                if tracking_id not in objetos_contados:
                    objetos_contados[tracking_id] = tipo_objeto
                    # Contar el objeto por tipo
                    conteo_clases[tipo_objeto] = conteo_clases.get(tipo_objeto, 0) + 1

            # Si es un vehículo, intentamos detectar la matrícula
            matricula_text = ""
            conf_matricula = 0
            mx1, my1, mx2, my2 = 0, 0, 0, 0
            if tipo_objeto in ["car", "bus", "motorcycle"]:  # Asumiendo que solo estos son vehículos
                vehiculo_roi = frame[y1:y2, x1:x2]  # Recortar la imagen solo en el área del vehículo
                matricula_results = model_matriculas.track(vehiculo_roi, stream=True, verbose=False)
                
                for mat_result in matricula_results:
                    mat_boxes = mat_result.boxes
                    for mat_box in mat_boxes:
                        mx1, my1, mx2, my2 = mat_box.xyxy[0]
                        mx1, my1, mx2, my2 = int(mx1), int(my1), int(mx2), int(my2)
                        
                        # Convertir a imagen y usar OCR para texto
                        matricula_roi = vehiculo_roi[my1:my2, mx1:mx2]
                        if matricula_roi is not None:
                            matricula_text = pytesseract.image_to_string(matricula_roi, config='--oem 3 --psm 8')
                            # ocr_results = reader.readtext(matricula_roi, detail=0)
                            # matricula_text = ocr_results[0] if ocr_results else ""
                            conf_matricula = math.ceil((mat_box.conf[0] * 100)) / 100
                        else:
                            matricula_text = ""
                            conf_matricula = 0
                        cv2.rectangle(frame, (mx1 + x1, my1 + y1), (mx2 + x1, my2 + y1), (0, 255, 0), 2)

            # Guardar resultados en el CSV
            csv_writer.writerow([frame_count, tipo_objeto, confianza, tracking_id, x1, y1, x2, y2, matricula_text, conf_matricula, mx1, my1, mx2, my2, matricula_text])
            
            # Dibujar en la imagen, incluyendo el ID de seguimiento si está disponible
            cv2.putText(frame, f"{tipo_objeto} {tracking_id} {confianza}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            if matricula_text:
                cv2.putText(frame, f"Matrícula: {matricula_text}", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Eliminar IDs no detectados en este fotograma del conjunto de IDs detectados
    for tid in list(objetos_contados.keys()):
        if tid not in detected_ids:
            del objetos_contados[tid]  # Eliminar el ID del conteo si no fue detectado

    detected_ids.clear()  # Limpiar el conjunto de IDs detectados para el siguiente fotograma

    # Mostrar el video con las detecciones
    cv2.imshow('Detección y Seguimiento', frame)
    out.write(frame)

    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Resumen de conteo de clases
print("Conteo total de clases detectadas:")
for clase, conteo in conteo_clases.items():
    print(f"{clase}: {conteo}")

# Cerrar todo
cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()


Conteo total de clases detectadas:
bus: 1
car: 44
person: 14
motorcycle: 1
skateboard: 1
